In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px

from scipy import stats
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA
import plotly.express as px

from scipy import stats
from pmdarima import auto_arima
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [6]:
df = pd.read_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Dataset and Code files/Tomato_Price_Transformed.xlsx',index_col='Date')

In [8]:
df.index = pd.to_datetime(df.index, format='%m/%Y')

In [10]:
df

,W.sale Avg.Price,Retail Avg. Price
Date,,
2000-01-01,603,687
2000-02-01,444,525
2000-03-01,392,479
2000-04-01,240,320
2000-05-01,736,823
...,...,...
2024-08-01,3308,5832
2024-09-01,4063,7113
2024-10-01,6296,9429


In [12]:
# Check NaN counts for all columns
nan_counts = df.isna().sum()
print("NaN counts per column:")
print(nan_counts)

# Iterate through the columns and find indices of NaN values
for column in df.columns:
    if df[column].isnull().any():  # Check if the column has any NaNs
        nan_indices = df[df[column].isnull()].index.tolist()
        print(f"\nNaN values found in column '{column}' at indices: {nan_indices}")

NaN counts per column:
W.sale Avg.Price     0
Retail Avg. Price    0
dtype: int64


In [20]:
zero_counts = (df == 0).sum()
print("Zero counts per column:")
print(zero_counts)

print("\nIndices of zero values per column:")
if (df['W.sale Avg.Price'] == 0).any():  # Check if the column has any zeros
    zero_indices = df[df['W.sale Avg.Price'] == 0].index.tolist()
    print(f"Zero values found in column 'W.sale Avg.Price' at indices: {zero_indices}")
else:
    print(f"No zero values found in column 'W.sale Avg.Price'.")

Zero counts per column:
W.sale Avg.Price     20
Retail Avg. Price    82
dtype: int64

Indices of zero values per column:
Zero values found in column 'W.sale Avg.Price' at indices: [Timestamp('2001-01-01 00:00:00'), Timestamp('2001-02-01 00:00:00'), Timestamp('2001-03-01 00:00:00'), Timestamp('2001-04-01 00:00:00'), Timestamp('2001-05-01 00:00:00'), Timestamp('2001-06-01 00:00:00'), Timestamp('2001-07-01 00:00:00'), Timestamp('2001-08-01 00:00:00'), Timestamp('2001-09-01 00:00:00'), Timestamp('2002-05-01 00:00:00'), Timestamp('2002-06-01 00:00:00'), Timestamp('2002-07-01 00:00:00'), Timestamp('2002-08-01 00:00:00'), Timestamp('2005-07-01 00:00:00'), Timestamp('2020-04-01 00:00:00'), Timestamp('2020-05-01 00:00:00'), Timestamp('2020-06-01 00:00:00'), Timestamp('2020-07-01 00:00:00'), Timestamp('2020-08-01 00:00:00'), Timestamp('2021-06-01 00:00:00')]


In [22]:
# Define the column name
col = 'W.sale Avg.Price'

# Function to calculate the average of same month in previous years with min_periods
def calculate_month_average_with_min_periods(row_date, data_series, min_periods=1):
    target_month = row_date.month
    previous_years_data = []
    
    for year_offset in range(1, 6):
        previous_year = row_date.year - year_offset
        try:
            previous_date = pd.Timestamp(year=previous_year, month=target_month, day=1)
            if previous_date in data_series.index and not pd.isna(data_series[previous_date]) and data_series[previous_date] != 0:
                previous_years_data.append(data_series[previous_date])
        except:
            continue
    
    # Calculate mean only if number of data points >= min_periods
    if len(previous_years_data) >= min_periods:
        return np.mean(previous_years_data)
    else:
        return np.nan

# Replace zeros with NaN
df[col] = df[col].replace(0, np.nan)

# Create a copy of the column with NaN values
filled_values = df[col].copy()

# For each NaN value, calculate the replacement
for date in df[df[col].isna()].index:
    filled_values[date] = calculate_month_average_with_min_periods(date, df[col], min_periods=1)

# Update the column with the filled values
df[col] = filled_values

# If there are still NaN values (no relevant history available), use overall mean
if df[col].isna().any():
    overall_mean = df[col].mean()
    df[col] = df[col].fillna(overall_mean)
    print(f"Some values filled with overall mean: {overall_mean}")

# Display the first few rows to verify


In [24]:
df

,W.sale Avg.Price,Retail Avg. Price
Date,,
2000-01-01,603.0,687
2000-02-01,444.0,525
2000-03-01,392.0,479
2000-04-01,240.0,320
2000-05-01,736.0,823
...,...,...
2024-08-01,3308.0,5832
2024-09-01,4063.0,7113
2024-10-01,6296.0,9429


In [34]:
df = df.reset_index()

In [28]:
df = df.drop(columns=['Retail Avg. Price'])

In [36]:
df.to_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Dataset and Code files/Tomato filled values.xlsx')


In [38]:
df = df.set_index('Date')

In [40]:
df

,W.sale Avg.Price
Date,
2000-01-01,603.0
2000-02-01,444.0
2000-03-01,392.0
2000-04-01,240.0
2000-05-01,736.0
...,...
2024-08-01,3308.0
2024-09-01,4063.0
2024-10-01,6296.0


In [42]:
estimated_df = pd.read_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/1. Estimated Data.xlsx',index_col='Date')

In [44]:
estimated_df

,Temperature,Inflation_rate,Growth_rate,Log_MS
Date,,,,
1991-01-01,-1.6,16.091935,4.644682,14.764768
1991-02-01,0.4,15.428546,7.792836,14.783057
1991-03-01,1.4,13.559333,8.108108,14.793190
1991-04-01,-0.1,12.222223,0.811771,14.817024
1991-05-01,0.8,12.087911,-1.981179,14.836756
...,...,...,...,...
2022-08-01,-0.2,6.998158,-0.679758,19.165064
2022-09-01,0.1,7.414216,3.320463,19.166083
2022-10-01,-0.7,6.767372,-4.074074,19.176894


In [46]:
# Start with df as the base
new_df = df.copy()

# Align estimated_df to df's index and join the columns
# Only rows with matching indices will be added
estimated_aligned = estimated_df.loc[estimated_df.index.intersection(df.index)]

# Add the columns to new_df
new_df = new_df.join(estimated_aligned)

In [52]:
new_df

,W.sale Avg.Price,Temperature,Inflation_rate,Growth_rate,Log_MS
Date,,,,,
2000-01-01,603.0,0.8,2.619042,9.043591,16.208273
2000-02-01,444.0,-1.8,3.614462,24.324022,16.223453
2000-03-01,392.0,-1.4,4.830924,30.215964,16.235144
2000-04-01,240.0,-0.1,5.542171,27.288814,16.261410
2000-05-01,736.0,0.2,5.011930,38.194843,16.267717
...,...,...,...,...,...
2022-08-01,2511.0,-0.2,6.998158,-0.679758,19.165064
2022-09-01,3837.0,0.1,7.414216,3.320463,19.166083
2022-10-01,4450.0,-0.7,6.767372,-4.074074,19.176894


In [50]:
new_df = new_df.dropna()


In [56]:
new_df = new_df.reset_index()

In [54]:
zero_counts = (new_df == 0).sum()
print("Zero counts per column:")
print(zero_counts)

print("\nIndices of zero values per column:")
# Iterate through the columns and find indices of zero values
for column in new_df.columns:
    if (new_df[column] == 0).any():  # Check if the column has any zeros
        zero_indices = new_df[new_df[column] == 0].index.tolist()
        print(f"Zero values found in column '{column}' at indices: {zero_indices}")

Zero counts per column:
W.sale Avg.Price     0
Temperature         14
Inflation_rate       0
Growth_rate          2
Log_MS               0
dtype: int64

Indices of zero values per column:
Zero values found in column 'Temperature' at indices: [Timestamp('2000-12-01 00:00:00'), Timestamp('2001-07-01 00:00:00'), Timestamp('2001-08-01 00:00:00'), Timestamp('2005-02-01 00:00:00'), Timestamp('2006-04-01 00:00:00'), Timestamp('2006-09-01 00:00:00'), Timestamp('2007-01-01 00:00:00'), Timestamp('2011-09-01 00:00:00'), Timestamp('2012-09-01 00:00:00'), Timestamp('2014-04-01 00:00:00'), Timestamp('2015-03-01 00:00:00'), Timestamp('2016-05-01 00:00:00'), Timestamp('2017-07-01 00:00:00'), Timestamp('2018-01-01 00:00:00')]
Zero values found in column 'Growth_rate' at indices: [Timestamp('2003-10-01 00:00:00'), Timestamp('2020-03-01 00:00:00')]


In [58]:
new_df = new_df.drop(columns=['Temperature'])


In [60]:
new_df.to_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Dataset and Code files/Tomato Joined.xlsx')


In [62]:
data = pd.read_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Dataset and Code files/Tomato Joined.xlsx',index_col='Date')

In [64]:
data

,Unnamed: 0,W.sale Avg.Price,Inflation_rate,Growth_rate,Log_MS
Date,,,,,
2000-01-01,0,603.0,2.619042,9.043591,16.208273
2000-02-01,1,444.0,3.614462,24.324022,16.223453
2000-03-01,2,392.0,4.830924,30.215964,16.235144
2000-04-01,3,240.0,5.542171,27.288814,16.261410
2000-05-01,4,736.0,5.011930,38.194843,16.267717
...,...,...,...,...,...
2022-08-01,271,2511.0,6.998158,-0.679758,19.165064
2022-09-01,272,3837.0,7.414216,3.320463,19.166083
2022-10-01,273,4450.0,6.767372,-4.074074,19.176894


In [66]:
data = data.drop(columns=['Unnamed: 0'])


In [68]:
print("Missing values:", data.isnull().sum())

Missing values: W.sale Avg.Price    0
Inflation_rate      0
Growth_rate         0
Log_MS              0
dtype: int64


In [70]:
data

,W.sale Avg.Price,Inflation_rate,Growth_rate,Log_MS
Date,,,,
2000-01-01,603.0,2.619042,9.043591,16.208273
2000-02-01,444.0,3.614462,24.324022,16.223453
2000-03-01,392.0,4.830924,30.215964,16.235144
2000-04-01,240.0,5.542171,27.288814,16.261410
2000-05-01,736.0,5.011930,38.194843,16.267717
...,...,...,...,...
2022-08-01,2511.0,6.998158,-0.679758,19.165064
2022-09-01,3837.0,7.414216,3.320463,19.166083
2022-10-01,4450.0,6.767372,-4.074074,19.176894


## Feature Engineering for Time Series Modeling

In [73]:
# Add temporal features
data['Month'] = data.index.month
data['Year'] = data.index.year
data['Quarter'] = data.index.quarter

# Add lagged features
for col in ['W.sale Avg.Price','Inflation_rate','Growth_rate', 'Log_MS']:
    data[f'{col}_1M_Lag'] = data[col].shift(1)
    data[f'{col}_3M_Lag'] = data[col].shift(3)

# Add rolling statistics
data['Price_3M_Rolling_Mean'] = data['W.sale Avg.Price'].rolling(window=3).mean()
data['Price_6M_Rolling_Mean'] = data['W.sale Avg.Price'].rolling(window=6).mean()
data['Price_3M_Rolling_Std'] = data['W.sale Avg.Price'].rolling(window=3).std()
data['Price_6M_Rolling_Std'] = data['W.sale Avg.Price'].rolling(window=6).std()

# Create interaction features
data['Inflation_Growth_Interaction'] = data['Inflation_rate'] * data['Growth_rate']

# Add seasonal decomposition features
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(data['W.sale Avg.Price'], model='multiplicative', period=12)
data['seasonal_component'] = decomposition.seasonal

# Drop rows with NaN values
data_processed = data.dropna()

# Updated exogenous variables after feature engineering
exog_enhanced = data_processed.drop(['W.sale Avg.Price'], axis=1)
target_processed = data_processed['W.sale Avg.Price']


In [75]:
data_processed

,W.sale Avg.Price,Inflation_rate,Growth_rate,Log_MS,Month,Year,Quarter,W.sale Avg.Price_1M_Lag,W.sale Avg.Price_3M_Lag,Inflation_rate_1M_Lag,...,Growth_rate_1M_Lag,Growth_rate_3M_Lag,Log_MS_1M_Lag,Log_MS_3M_Lag,Price_3M_Rolling_Mean,Price_6M_Rolling_Mean,Price_3M_Rolling_Std,Price_6M_Rolling_Std,Inflation_Growth_Interaction,seasonal_component
Date,,,,,,,,,,,,,,,,,,,,,
2000-06-01,1182.0,5.238095,38.171534,16.281510,6,2000,2,736.0,392.0,5.011930,...,38.194843,30.215964,16.267717,16.235144,719.333333,599.500000,471.221109,332.994745,199.946120,1.244693
2000-07-01,953.0,4.952828,13.306099,16.281959,7,2000,3,1182.0,240.0,5.238095,...,38.171534,27.288814,16.281510,16.261410,957.000000,657.833333,223.026904,363.031909,65.902813,1.386104
2000-08-01,944.0,3.990609,22.953398,16.288343,8,2000,3,953.0,736.0,4.952828,...,13.306099,38.194843,16.281959,16.267717,1026.333333,741.166667,134.886372,361.513716,91.598023,1.185297
2000-09-01,920.0,3.496508,41.795593,16.303125,9,2000,3,944.0,1182.0,3.990609,...,22.953398,38.171534,16.288343,16.281510,939.000000,829.166667,17.058722,321.577622,146.138613,1.137432
2000-10-01,3500.0,2.746003,47.315842,16.319555,10,2000,4,920.0,953.0,3.496508,...,41.795593,13.306099,16.303125,16.281959,1788.000000,1372.500000,1482.684053,1051.858118,129.929446,1.408927
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-08-01,2511.0,6.998158,-0.679758,19.165064,8,2022,3,2389.0,4426.0,6.707692,...,2.205323,19.721981,19.164157,19.147374,3242.000000,2826.500000,1373.139833,1476.515188,-4.757056,1.185297
2022-09-01,3837.0,7.414216,3.320463,19.166083,9,2022,3,2511.0,4826.0,6.998158,...,-0.679758,12.622150,19.165064,19.146480,2912.333333,3238.833333,803.104808,1323.604989,24.618631,1.137432
2022-10-01,4450.0,6.767372,-4.074074,19.176894,10,2022,4,3837.0,2389.0,7.414216,...,3.320463,2.205323,19.166083,19.164157,3599.333333,3739.833333,991.107630,1048.727117,-27.570773,1.408927


In [77]:
data_processed.to_excel('/Users/ashutoshanand/Documents/UGP- Sanjiv/Dataset and Code files/Tomato Processed data.xlsx')